## Grouping and Aggregating

### Aggregate Functions - these operators work on a set of values from mulyiple rows at once

- COUNT - counts the number of non-NULL values in a column,works on BIT column, no other does.
- SUM 
- AVG 
- MIN - can be used on character data as well as date and numeric
- MAX

AGGREGATE functions dont work on TEST,NTEST and IMAGE columns  
<span style="color: var(--vscode-foreground);">except for COUNT others dont work on special data types like hierarchyID</span>

COUNT can be used with \* to give the count of all the rows  
  
all the aggregate functions ignore null except for count when using with \*

  

SYNTAX

> SELECT \<AGG FUNC\>(\<COL1\>)  
> FROM \<TABLE\>

In [ ]:
SELECT COUNT(*) AS COUNTOFROWS,
    MAX(H.TotalDue) AS MaxTotal,
    MIN(H.TotalDue) AS MINTOTAL,
    SUM(H.TotalDue) AS SUMOFTOTAL,
    AVG(H.TotalDue) AS AVGTOTAL 
FROM Sales.SalesOrderHeader AS H;

In [ ]:
SELECT MIN(Name) AS MinName,
 MAX(Name) AS MaxName,
 MIN(SellStartDate) AS MinSellStartDate
FROM Production.Product;

In [ ]:
SELECT COUNT(*) AS CountOfRows,
 COUNT(Color) AS CountOfColor,
 COUNT(DISTINCT Color) AS CountOfDistinctColor
FROM Production.Product;

We can also have simple columns along wth aggregate functions as part of the same column.

### Exercise 7.1

Use the AdventureWorks2019 database to complete Questions 1 to 5. Switch to the WideWorldImporters database to complete this exercise.

1\. Write a query to determine the number of customers in the Sales.Customer table.

In [ ]:
SELECT COUNT(DISTINCT C.CustomerID)
FROM Sales.Customer AS C;

2\. Write a query that retrieves the total number of products ordered. Use the OrderQty column of the Sales.SalesOrderDetail table and the SUM function.

In [ ]:
SELECT SUM(OrderQty)
FROM Sales.SalesOrderDetail

3\. Write a query to determine the price of the most expensive product ordered. Use the UnitPrice column of the Sales.SalesOrderDetail table.

In [ ]:
SELECT MAX(D.UnitPrice)
FROM Sales.SalesOrderDetail AS D;

4\. Write a query to determine the average freight amount in the Sales. SalesOrderHeader table.

In [ ]:
SELECT AVG(H.Freight)
FROM Sales.SalesOrderHeader AS H;

5\. Write a query using the Production.Product table that displays the minimum, maximum, and average ListPrice.

In [ ]:
SELECT MIN(ListPrice),MAX(ListPrice),AVG(ListPrice) 
FROM Production.Product;

### GROUP BY

use when we want aggreagation on the basis of some common value, use with categorical data

### Grouping on columns

- for when we wanto calculate the sum of all the orders grouped by the date or grouped by the customer
- An additional column may be included in the select list, the column being the one we group the data on
- once you have included normal non aggregate columns in the select list we must group by on them, or you will get an error

SYNTAX

> SELECT \<AGG FUNC\>(\<COL1\>), \<COL2\>  
> <span style="color: var(--vscode-foreground);">FROM &lt;TABLE&gt;&nbsp;<br></span> <span style="color: var(--vscode-foreground);">GROUP BY &lt;COL2&gt;</span>

In [ ]:
SELECT CustomerID,SUM(TotalDue) AS TotalPerCustomer
FROM Sales.SalesOrderHeader as H 
GROUP BY CustomerID;

In [ ]:
SELECT TerritoryID,AVG(TotalDue) AS AveragePerTerritory
FROM Sales.SalesOrderHeader
GROUP BY TerritoryID;

### GROUP ON EXPRESSIONS

- we can group on soem expression applied on some column as well

In [2]:
SELECT COUNT(*) AS CountOfOrders, YEAR(OrderDate) AS OrderYear
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate);

(4 rows affected)

Total execution time: 00:00:00.048

CountOfOrders,OrderYear
14182,2013
11761,2014
1607,2011
3915,2012


- you must group by on the same expressio in the selelct list else the result will ont be what you expected

EXERCISE 7.2

Use the AdventureWorks2019 database to complete Questions 1 to 4. Switch to the WideWorldImporters database to complete the exercise.

1\. Write a query that shows the total number of items ordered for each product. Use the Sales.SalesOrderDetail table to write the query.

In [ ]:
SELECT SUM(D.OrderQty),COUNT(D.OrderQty),p.Name
FROM Sales.SalesOrderDetail AS D
INNER JOIN Production.Product AS P ON P.ProductID=D.ProductID
GROUP BY P.Name

2\. Write a query using the Sales.SalesOrderDetail table that displays a count of the detail lines for each SalesOrderID

In [ ]:
SELECT D.SalesOrderID, COUNT(D.SalesOrderID),COUNT(*)
FROM Sales.SalesOrderDetail AS D 
GROUP BY D.SalesOrderID

3\. Write a query using the Production.Product table that lists a count of the products in each product line.

In [ ]:
SELECT COUNT(P.ProductID),P.ProductLine
FROM Production.Product AS P 
GROUP BY P.ProductLine

4\. Write a query that displays the count of orders placed by year for each customer using the Sales.SalesOrderHeader table.

In [ ]:
SELECT YEAR(H.OrderDate),COUNT(*),CustomerID
FROM Sales.SalesOrderHeader AS H 
GROUP BY YEAR(H.OrderDate), CustomerID
ORDER BY CustomerID;
SELECT YEAR(H.OrderDate),COUNT(*),CustomerID
FROM Sales.SalesOrderHeader AS H 
GROUP BY CustomerID, YEAR(H.OrderDate)
ORDER BY CustomerID;

5\. Switch to WideWorldImporters. Write a SELECT statement that displays a count of CountryNames for each Continent. Use the Application.Countries table. Alias the COUNT as CountCountries.

In [1]:
SELECT C.Continent, COUNT(C.CountryID)
FROM Application.Countries AS C 
GROUP BY C.Continent

(7 rows affected)

Total execution time: 00:00:00.056

Continent,(No column name)
Africa,50
Asia,46
Europe,43
North America,23
Oceania,13
Seven seas (open ocean),3
South America,12


### ORDER BY

- special considerations for order by when using with group by
- if a non aggregated function is used to order by it must be grouped on
- the following code will throw error
- the thing we are ordering on can be a aggregate function, then it doesnt have to be in the group by

In [5]:
SELECT H.CustomerId, SUM(H.TotalDue) 
FROM Sales.SalesOrderHeader as H 
GROUP BY CustomerID
ORDER BY TerritoryID;

: Msg 8127, Level 16, State 1, Line 4
Column "Sales.SalesOrderHeader.TerritoryID" is invalid in the ORDER BY clause because it is not contained in either an aggregate function or the GROUP BY clause.

Total execution time: 00:00:00.001

- the espression we group on doesnt need to be in the sekect list

In [ ]:
SELECT H.CustomerID, SUM(TotalDue)
FROM Sales.SalesOrderHeader AS H
GROUP BY H.CustomerID
ORDER BY MAX(H.TotalDue) DESC;

### WHERE CLAUSE

- the where clause may not contain any aggregate expression
- where clause is used to eliminate rows before the groupings and aggregations
- to filter after the groupings and aggrigations use the having caluse
- not necessary to ave the column we applied here on to be in the select list

In [ ]:
SELECT H.CustomerID, SUM(H.TotalDue)
FROM Sales.SalesOrderHeader AS H 
WHERE H.TerritoryID IN (5,6)
GROUP BY H.CustomerID;

SELECT H.CustomerID, SUM(H.TotalDue), H.TerritoryID
FROM Sales.SalesOrderHeader AS H  
WHERE H.TerritoryID IN (5,6)
GROUP BY H.CustomerID, H.TerritoryID;

### HAVING CLAUSE

- to eliminate rows based on aggreagate function after the grouping
- may contains aggregate experssions that may or may not be included in the select list
- we can have non aggregate columns in the having clause as long as we also group on it, allowing us to filter some of hte groups 
- under the hood the data base engine may move the having clauses to the where clause conditions because thatst way more efficient
- try to have criterions with non aggregate columns in the where clause

In [ ]:
SELECT SUM(H.TotalDue)
FROM Sales.SalesOrderHeader AS H 
GROUP BY H.CustomerID
HAVING COUNT(H.CustomerID) >= 10;--TOTAL DUE FOR CUSTOMERS THAT HAVE ORDERED MORE THAN 10 TIMES
--THIS WAS AN EXAMPLE OF TOTAL DUE FOR EACH CUSTORE IF THEY HAVE ORDERED MORE THAN 10 TIMES
-- WE CAN ALSO DO SOMETHING LIKE TOTAL DUE FOR EACH CUSTOMER IF THEIR TOTAL DUE EXCEEDS 10000 LIKE SO
SELECT SUM(H.TotalDue)
FROM Sales.SalesOrderHeader AS H 
GROUP BY H.CustomerID
HAVING SUM(H.TotalDue) >= 10000;--TOTAL DUE FOR CUSTOMERS THAT OWE MORE HTAN 10000

SELECT CustomerID,SUM(TotalDue) AS TotalPerCustomer
FROM Sales.SalesOrderHeader
GROUP BY CustomerID
HAVING CustomerID > 27858;
-- we can also do something like this, using non aggregate columns in the having clause, it is not recommended
-- keep criterions like this in the where clause

### order of operations

- FROM
- WHERE
- GROUP BY
- HAVING
- SELECT
- ORDER BY

### EXERCISE 7.3

### Use the AdventureWorks2019 database to complete Questions 1 to 4. Switch to the WideWorldAdventures database to complete this exercise.

1\. Write a query that returns a count of detail lines in the Sales. SalesOrderDetail table by SalesOrderID. Include only those sales that have more than three detail lines.

In [ ]:
SELECT COUNT(*), H.SalesOrderID
FROM Sales.SalesOrderDetail AS H 
GROUP BY H.SalesOrderID
HAVING COUNT(*)>3;

2\. Write a query that creates a sum of the LineTotal in the Sales. SalesOrderDetail table grouped by the SalesOrderID. Include only those rows where the sum exceeds 1000

In [ ]:
SELECT SUM(LineTotal), H.SalesOrderID
FROM Sales.SalesOrderDetail AS H 
GROUP BY H.SalesOrderID
HAVING SUM(H.LineTotal)>1000;

3\. Write a query that groups the products by ProductModelID along with a count. Display the rows that have a count that equals 1

In [ ]:
SELECT P.ProductModelID, COUNT(*)
FROM Production.Product AS P 
GROUP BY P.ProductModelID 
HAVING COUNT(*) = 1; 

4\. Change the query in Question 3 so that only the products with the color blue or red are included

In [ ]:
SELECT P.ProductModelID, COUNT(*), P.Color
FROM Production.Product AS P 
WHERE P.Color IN ('Blue','Red')
GROUP BY P.ProductModelID, P.Color
HAVING COUNT(*) = 1; 